# Velkoz Stock Data Extraction and Ingestion API

Documentation is hard. So this notebook provides examples and descriptions of how to make use of the velkoz web extraction library to extract and write data to a database. This notebook specifically focuses on the stock data module of the library and the APIs that are available.

This is more of a use-guide (although some explanation of architecture / justifications will be done). It assumes that you have a base understanding of how velkoz classes are used to extract and ingest data. See the [sphinx documentation](https://github.com/MatthewTe/velkoz_web_data_extraction_library/tree/master/docs) about `base objects` for an outline. 



## Stock Price Data

Like all good stock data web scraping libraries, the velkoz library gets its time series price data from Yahoo Finance. Actually its even worse than that. The library shamelessly makes use of the existing [yfinance](https://pypi.org/project/yfinance/) python package to perform the actual heavy lifting of extracting of performing the request to Yahoo Finance servers. 

The price data web objects simply wrap this functionality in a way that is compatible with the velkoz framework of data ingestion. **Seriously, show some love to the [creator](https://github.com/ranaroussi/yfinance) of the yfinance packag, people like them are carrying the python quant community on its back.**


### Necessary Objects:
The two main objects that are used to extract and write data are:

* ##### StockPriceResponseObject: Extracting stock price data from the Yahoo Finance Server
* ##### StockPriceDataIngestionEngine: Writing extracted data to the database


In [1]:
# Importing necessary velkoz objects:
from velkoz_web_packages.objects_stock_data.objects_stock_price.web_objects_stock_price import NASDAQStockPriceResponseObject
from velkoz_web_packages.objects_stock_data.objects_stock_price.ingestion_engines_stock_price import StockPriceDataIngestionEngine

### Extracing Raw Price Timeseries:

Data is extracted through the creation of a PriceResponse Object:

In [2]:
# Creating ResponseObject that makes call to Yahoo Finance Servers for max price history data:
tsla = NASDAQStockPriceResponseObject("TSLA")
aapl = NASDAQStockPriceResponseObject("AAPL")
icln = NASDAQStockPriceResponseObject("ICLN")

In [3]:
# Data can be accessed through intenal StockPriceResponseObject parameters:
print("Ticker:", tsla._ticker) # str
print("Time Method Was Called:" ,tsla._initialized_time) # float
print(tsla._price_history_full) # DataFrame

Ticker: TSLA
Time Method Was Called: 2020-11-27 02:02:27.324086
              open    high     low   close    volume  dividends  stock_splits
Date                                                                         
2010-06-29    3.80    5.00    3.51    4.78  93831500          0           0.0
2010-06-30    5.16    6.08    4.66    4.77  85935500          0           0.0
2010-07-01    5.00    5.18    4.05    4.39  41094000          0           0.0
2010-07-02    4.60    4.62    3.74    3.84  25699000          0           0.0
2010-07-06    4.00    4.00    3.17    3.22  34334500          0           0.0
...            ...     ...     ...     ...       ...        ...           ...
2020-11-19  492.00  508.61  487.57  499.27  62475300          0           0.0
2020-11-20  497.99  502.50  489.06  489.61  32807300          0           0.0
2020-11-23  503.50  526.00  501.79  521.85  50260300          0           0.0
2020-11-24  540.40  559.99  526.20  555.38  53648500          0           0.0


### Writing Extracted Price Data To Database:

The data extracted via a PriceResponse Object then needs to be added to an instance of an Ingestion Engine, which is then used to write data to the database:

In [4]:
# Creating Ingestion Engine:
example_uri = "sqlite:///:memory:" # In-memeory SQLITE database for demo purposes.
stock_ingestion_engine = StockPriceDataIngestionEngine(example_uri, tsla)

In [5]:
# tsla PriceResponse Object is now stored within the Ingestinon Engines' internal que:
print(stock_ingestion_engine._WebPageResponseObjs)

[yfinance.Ticker object <TSLA>]


In [6]:
# Ingestion Engine Que can be modified as such:

# Empty Web Object Que
stock_ingestion_engine._purge_web_obj_que() 
print(stock_ingestion_engine._WebPageResponseObjs)

# Insert Web Object into Que:
stock_ingestion_engine._insert_web_obj(tsla)
stock_ingestion_engine._insert_web_obj(aapl)
stock_ingestion_engine._insert_web_obj(icln)
print(stock_ingestion_engine._WebPageResponseObjs)

[]
[yfinance.Ticker object <TSLA>, yfinance.Ticker object <AAPL>, yfinance.Ticker object <ICLN>]


In [7]:
# Actually Writing Price data to the database:
stock_ingestion_engine._write_web_objects() # Echo is set to True

# Writing data to database should empty the Ingestion Engine Que:
print(stock_ingestion_engine._WebPageResponseObjs)

2020-11-27 02:02:29,149 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-27 02:02:29,151 INFO sqlalchemy.engine.base.Engine ()
2020-11-27 02:02:29,153 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-27 02:02:29,154 INFO sqlalchemy.engine.base.Engine ()
2020-11-27 02:02:29,157 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("TSLA_price_history")
2020-11-27 02:02:29,167 INFO sqlalchemy.engine.base.Engine ()
2020-11-27 02:02:29,170 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("TSLA_price_history")
2020-11-27 02:02:29,171 INFO sqlalchemy.engine.base.Engine ()
2020-11-27 02:02:29,174 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "TSLA_price_history" (
	"Date" DATETIME, 
	open FLOAT, 
	high FLOAT, 
	low FLOAT, 
	close FLOAT, 
	volume BIGINT, 
	dividends BIGINT, 
	stock_splits FLOAT
)


2020-11-27 02:02:29,175 INFO sqlalchemy.engine.base.Engine ()
2020-11-27 0

## Fund Top 10 Holdings Data:

The top 10 Fund holdings objects may only be of marginal use but it was low hanging fruit in terms of adding functionality to the library. It sends a request to Yahoo Finance servers (hand written this time, no finance package!) to extract the top-1o holdings data contained in the “holdings” tab of a fund.

### Necessary Objects:
The two main objects that are used to extract and write data are:

* ##### FundHoldingsResponseObject: Extracting fund holdings data from the Yahoo Finance Server
* ##### FundHoldingsDataIngestionEngine: Writing extracted data to the database.

In [8]:
# Importing velkoz modules:
from velkoz_web_packages.objects_stock_data.objects_fund_holdings.web_objects_fund_holdings import NASDAQFundHoldingsResponseObject
from velkoz_web_packages.objects_stock_data.objects_fund_holdings.ingestion_engines_fund_holdings import FundHoldingsDataIngestionEngine

### Extracting Raw Fund Holdings Data:
Data is extracted through the creation of a FundHoldingsResponse Object:

In [9]:
# Creating Response Object that conatins data as internal parameters:
icln = NASDAQFundHoldingsResponseObject('ICLN')
voo = NASDAQFundHoldingsResponseObject('VOO')
spy = NASDAQFundHoldingsResponseObject('SPY')

In [10]:
# Accessing parameters contained within Response Object:
print("Fund Ticker:", icln._ticker) # string
print("Generated Yahoo Finance URL:", icln._yhfinance_url) # String
# print("HTML Tag Containing Holdings Data:", icln._holdings_tbl)
print(icln._holdings_data)

Fund Ticker: ICLN
Generated Yahoo Finance URL: https://finance.yahoo.com/quote/ICLN/holdings
                                       name  percent_holdings
symbol                                                       
FSLR                        First Solar Inc              5.28
00968              Xinyi Solar Holdings Ltd              5.04
ORSTED                           Orsted A/S              4.86
CEN.NZ                   Contact Energy Ltd              4.82
MEL.NZ                  Meridian Energy Ltd              4.82
VWS                 Vestas Wind Systems A/S              4.74
VER                              Verbund AG              4.68
SGRE.BC  Siemens Gamesa Renewable Energy SA              4.58
ENPH                     Enphase Energy Inc              4.41
ORA                  Ormat Technologies Inc              4.07


### Writing Extracted Price Data To Database:

The data extracted via a FundHoldingsResponse Object then needs to be added to an instance of an Ingestion Engine, which is then used to write data to the database:

In [11]:
# Creating Fund Holdings Ingestion Engine w/ In-Memory database:
holdings_ingestion_eninge = FundHoldingsDataIngestionEngine(example_uri)

In [12]:
# Showing Examples of all internal methods to add/remove data from Ingestion Engines' internal data que:

# Adding an object to the que:
holdings_ingestion_eninge._insert_web_obj(icln)
print(holdings_ingestion_eninge._WebPageResponseObjs)

# Purging the que:
holdings_ingestion_eninge._purge_web_obj_que()
print(holdings_ingestion_eninge._WebPageResponseObjs)

# Re-Populating Que:
holdings_ingestion_eninge._insert_web_obj(icln)
holdings_ingestion_eninge._insert_web_obj(voo)
holdings_ingestion_eninge._insert_web_obj(spy)
print(holdings_ingestion_eninge._WebPageResponseObjs)

[WebObject(https://finance.yahoo.com/quote/ICLN/holdings_2020-11-27 02:02:32.323764)]
[]
[WebObject(https://finance.yahoo.com/quote/ICLN/holdings_2020-11-27 02:02:32.323764), WebObject(https://finance.yahoo.com/quote/VOO/holdings_2020-11-27 02:02:35.711480), WebObject(https://finance.yahoo.com/quote/SPY/holdings_2020-11-27 02:02:39.152541)]


In [14]:
# Writing data to database (should empty que as a result):
holdings_ingestion_eninge._write_web_objects() # Echo Is on.
print("QUE After Ingestion:", holdings_ingestion_eninge._WebPageResponseObjs)

QUE After Ingestion: []


In [ ]:
# TODO: Add Stock DB Summary Objects to Notebooks.